In [1]:
import os
from selenium import webdriver
import time
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import networkx as nx
from concurrent.futures import ThreadPoolExecutor
from math import ceil
from functools import wraps

 네이버에서 서적을 검색하고 해당 서적 설명 페이지에서 서평 내용 크롤링 하기

In [2]:
# 재시도 데코레이션 함수
def retry(max_retries=3, delay_in_seconds=2):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            retries = 0
            while retries < max_retries:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    retries += 1
                    if retries < max_retries:
                        time.sleep(delay_in_seconds)
                    else:
                        raise e
        return wrapper
    return decorator

In [3]:
# 교보 문고 홈페이지에서 특정 도서에 대해 검색하기
@retry(max_retries=3, delay_in_seconds=2)
def search_book(driver,name):
# url안에서 자체로 도서를 인기순으로 정렬     
    base_url = 'https://search.kyobobook.co.kr/search?keyword={}&gbCode=TOT&target=total'
    driver.get(base_url.format(name))
    wait = WebDriverWait(driver, 10)
# 검색했을 때 책이 나오는 경우 3가지(두 개 이상 - 한 개 - 0 개)    
    try :
        book_link = wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="shopData_list"]/ul/li[1]/div[1]/div[2]/div[2]/div[1]/div/a')))
        book_link.click()
    except:
        book_link = wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="shopData_list"]/ul/li/div[1]/div[2]/div[2]/div[1]/div/a')))
        book_link.click()
        
    

In [4]:
@retry(max_retries=3, delay_in_seconds=2)
def collect_review(driver, name) :
    wait = WebDriverWait(driver, 10)
    comment = ''
    review_tag = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,'div.intro_bottom  div.info_text')))

    if len(review_tag) == 1 :
        comment = review_tag[0].text
    elif len(review_tag) >= 2 :
        for review in review_tag:
            comment += review.text
    else:
        comment = None
    return comment

In [5]:
# 그룹 이름 따로 관리
df = pd.read_excel('list_of_books_by_groups.xlsx')
group_name_list = df.columns.tolist()
df.shape

(200, 90)

In [6]:
# text_ranking.ipynb 파일의 함수를 이용하기 위해 파일 로드
%run text_ranking2.ipynb

403
['스', '사피', '인간', '인류', '출', '간', '책', '저자', '서문', '더', '특별', '이제', '할', '이해', '한다', '인공', '지능', '시대', '세상', '동료', '나아갈', '마음', '전하', '통찰', '명실', '역사', '인문', '가로지르는', '미래', '작', '호소', '풍부한', '없었다', '호모', '읽고', '있어', '시장', '제작', '추천', '지식', '리', '사랑', '통해', '수', '글', '종횡무진', '주년', '이후', '강조', '상부']


In [7]:
# text_ranking.ipynb 파일에서 필요한 파일을 정의
f=open('stopwords.txt','r',encoding='utf-8')
stop_words = f.read().splitlines()

In [8]:
# 검색의 정확도를 높이기 위해 제목을 가공하는 코드
def truncate_after_delimiters(text, delimiters):
# 소괄호 안에 있는 문자열 제거
    while "(" in text and ")" in text:
        start = text.find("(")
        end = text.find(")")
        text = text[:start] + text[end+1:]
# 특정 구분자 이후 문자열 제거
    for delimiter in delimiters:
        if delimiter in text:
            text = text.split(delimiter)[0] 
            break
    return text

In [9]:
# 드라이버 재시작 함수
def restart_driver(driver):
    driver.quit()
    driver = webdriver.Chrome()
    return driver

In [10]:
# 오류나면 진행 내용 저장 함수
def save_intermediate_result_to_excel(data, filename):
    df_temp = pd.DataFrame(data)
    df_temp.to_excel(filename)

In [11]:
# 데이터 프레임에서 그룹별로 선택 후 그룹 안에서 각각의 책 추출
# 추출한 책의 서평 크롤링 후, 텍스트 랭크(or LDA) 적용 
# 책마다 추출 된 키워드들을 하나의 셀에 저장 하나의 컬럼은 하나의 그룹
def crawl_and_extract_keywords_for_threads(index_range):
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    
    restart_interval = 1
    
    thread_results ={}
    
    error_list_search_book = []  
    error_list_collect_review = []
    
    delimiters = [':','=','/','-']
    
    for i in index_range:
        column = df.iloc[:,i]
        group_name = group_name_list[i]
        keywords_list = []
        
        for n in range(20):
            r_name = column.iloc[n+1]
            name = truncate_after_delimiters(r_name,delimiters)
            try:
                search_book(driver,name)
                time.sleep(1)
# 책이 없는 경우 다음 반복으로 넘어가야 하므로 search_book 함수 밖에서 실행            
            except Exception as e:
                print(f"Error in search_book for the book: {i} - {name}")
                error_list_search_book.append(name)
                keywords_list.append(np.nan)
                continue
            try:
                comment = collect_review(driver,name)
# 출판사 서평이 없거나, 서평을 긁는데 문제가 생긴 경우
            except Exception as e:
                print(f"Error in collection_review for the book: {i} - {name}")
                error_list_collect_review.append(name)
                keywords_list.append(np.nan)
                continue
# text_ranking 실행
            try:
                graph = nx.Graph()
                words = preprocess_text(comment, stop_words)
                tfidf_scores = compute_tfidf_scores_v2(comment, stop_words, preprocess_text)
                word_graph_with_tfidf(words, tfidf_scores, graph)
                keywords = text_ranking(graph)
                keywords_list.append(keywords[0:30])
                print(str(i)+"-"+str(n+1)+'완료')
                print(keywords)
            except Exception as e:
                    print(f"Error processing book : {i} - {name}")
                    save_intermediate_results_to_excel(thread_result,"intermediate_result.xlsx")
            
        thread_results[group_name] = keywords_list
        time.sleep(2)
        
        if (i+1) % restart_interval == 0:
            driver = restart_driver(driver)
                
    return (thread_results, error_list_search_book, error_list_collect_review)

In [12]:
def parallel_crawling_and_extraction_with_threaded_drivers():
    
    num_threads = 9
    tasks_per_thread = ceil(90 / num_threads)
    index_ranges = [range(i, i + tasks_per_thread) for i in range(0, 90, tasks_per_thread)]
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        results = list(executor.map(crawl_and_extract_keywords_for_threads, index_ranges))
   
    merged_results = {}
    merged_error_list_search = []
    merged_error_list_collect = []
    
    for result in results:
        merged_results.update(result[0])
        merged_error_list_search.extend(result[1])
        merged_error_list_collect.extend(result[2])

    return merged_results, merged_error_list_search, merged_error_list_collect 

In [ ]:
result, error_search, error_collect = parallel_crawling_and_extraction_with_threaded_drivers()

30-1완료
['책', '독자', '간', '개정', '출', '이야기', '하는', '갖게', '인문학', '나누었던', '대', '바꾼', '한', '않은', '발음', '아성', '지난', '뒤', '기초', '시킨', '받는', '모든', '후', '하지', '처음', '하기도', '깨지지', '순간', '상식', '탄생', '평가', '지적', '많은', '만나', '시작', '중학생', '베스트셀러', '독학', '증보', '읽게', '열광하고', '하고', '사장', '이어', '판도', '데', '읽고', '오르는', '대표', '모임', '주관', '시대', '판', '분야', '첫', '세대', '자리', '어렵고', '판이', '수년', '얕', '채', '독서', '수백만', '딱딱하던', '선거', '잡은', '편', '필수', '도서', '꿈쩍', '맞게', '낯설고', '낡은', '시리즈', '않았는지', '도움', '직장인', '읽는', '목말', '프로그램', '쓰였다고', '대화', '불문', '옷', '넣었다', '어려운', '지대', '넓', '하게', '작가', '확인', '돌아왔다', '만', '바꿨다', '인문', '개정판', '갈아입고', '보게', '있던', '콘셉트', '재미있게', '위', '나왔다', '뉴스', '교양', '제로', '부', '되었다', '주체', '토론', '셀러', '남녀', '채워', '거칠고', '된', '꼼꼼하게', '테디', '신간', '보통', '유사', '얕이라는', '쉽고', '넓고', '수선', '노소', '낌', '매만지고', '될', '말', '누적', '사람', '시키고', '거리', '있도록', '부족했던', '얕은', '설명', '읽을', '유행', '스', '부드럽게', '부분', '수', '판매', '대한', '지식', '하며', '돌파', '기록', '해', '밀리언셀러', '더블']
80-1완료
['독서', '책', '법', '교육', '하는', '공부', '정

30-2완료
['책', '독자', '간', '개정', '출', '이야기', '하는', '갖게', '인문학', '나누었던', '대', '바꾼', '한', '않은', '발음', '아성', '지난', '뒤', '기초', '시킨', '받는', '모든', '후', '하지', '처음', '하기도', '깨지지', '순간', '상식', '탄생', '평가', '지적', '많은', '만나', '시작', '중학생', '베스트셀러', '독학', '증보', '읽게', '열광하고', '하고', '사장', '이어', '판도', '데', '읽고', '오르는', '대표', '모임', '주관', '시대', '판', '분야', '첫', '세대', '자리', '어렵고', '판이', '수년', '얕', '채', '독서', '수백만', '딱딱하던', '선거', '잡은', '편', '필수', '도서', '꿈쩍', '맞게', '낯설고', '낡은', '시리즈', '않았는지', '도움', '직장인', '읽는', '목말', '프로그램', '쓰였다고', '대화', '불문', '옷', '넣었다', '어려운', '지대', '넓', '하게', '작가', '확인', '돌아왔다', '만', '바꿨다', '인문', '개정판', '갈아입고', '보게', '있던', '콘셉트', '재미있게', '위', '나왔다', '뉴스', '교양', '제로', '부', '되었다', '주체', '토론', '셀러', '남녀', '채워', '거칠고', '된', '꼼꼼하게', '테디', '신간', '보통', '유사', '얕이라는', '쉽고', '넓고', '수선', '노소', '낌', '매만지고', '될', '말', '누적', '사람', '시키고', '거리', '있도록', '부족했던', '얕은', '설명', '읽을', '유행', '스', '부드럽게', '부분', '수', '판매', '대한', '지식', '하며', '돌파', '기록', '해', '밀리언셀러', '더블']
50-2완료
['책', '읽기를', '시기', '부모', '저학년', '한다',

80-4완료
['정치', '수', '민주주의', '할', '사회', '진보', '어른', '보수', '지적', '경제', '한다', '오메가', '이해', '선거', '있었다', '무관심', '역사', '없는', '생각', '대화', '편', '이야기', '있을까', '독재', '어린이', '하려는데', '입문', '지나', '채', '정당화', '성화', '하는', '여러', '가능하지', '어렵고', '앉아', '만남', '입장', '아는', '뉴스', '이제', '다다랐다', '복잡한', '현대', '낸', '실망하여', '풀어', '나오는', '집단', '본질', '이슈', '다르다다음', '무능력', '비유', '마주', '나눠', '성향', '위해', '세계', '정치인', '가장', '결정', '실현', '배꼽', '띠', '구분', '지식', '권', '분리', '하기도', '선택', '통해', '인문학', '호기심', '시민', '알파', '얕', '필요한', '집안', '후자', '현실', '경험', '할지', '뛰어넘었고', '이루어져', '카페', '어려운', '되는', '세상', '이후', '역시', '아빠', '목말', '이루는', '공부', '일컫는다', '사안', '자신감', '주제', '뿜뿜', '단순하게', '않다', '되어', '토론', '찾아', '걱정', '사장', '모르겠다는', '생산', '바꿀', '내', '살아가기', '개인', '반대', '체험', '간', '발견', '영역', '있다니', '가상', '재미있을', '넓', '친구', '어느덧', '권한', '엄마', '학교', '쉬', '보면', '명쾌하게', '관심', '초등학생', '뽑아야', '습득', '지도', '훌륭한', '도움', '빠른', '필요하다고', '한다고', '이유', '된', '정리', '웃픈', '준다', '속도', '기우', '가졌으며', '였다', '하기', '모습', '여행', '이자', '팬', '신비한', '부패', '기다려요',

50-5완료
['책', '설민석', '이야기', '권', '못', '한', '땅', '읽어', '방송', '인생', '책방', '추려', '요즘', '삶', '닿는', '샀지만', '친절함이다', '인문학', '그동안', '여러', '읽지', '선정', '사람', '가장', '중', '보고', '드립니다는', '다루었던', '벅찬', '되어줄', '입', '가슴', '지금', '드립니다에서', '바구니', '사야겠다고', '내용', '역사', '계기', '넣어', '인간', '한다', '구석구석', '인문', '유산', '여행', '같은', '있는', '재미있게', '풀어내고자', '교양', '보석', '페스트', '함축', '경연', '인류', '경험', '가진', '꽂아', '하지', '푼', '건네는', '고전', '서가에', '떠나는', '하고', '능력', '넣은', '휩쓸고', '두었거나', '쉽고', '초대장', '담화', '불어', '강독', '리처드', '토대', '체험', '만난', '맥락', '진행', '공부', '기쁨', '도킨스', '성장', '통해', '나누는', '해석', '빠져드는', '둔', '이기', '두껍고', '시대', '유전자', '흥미롭다', '코로나', '독자', '넘기며', '소설', '종말', '유발', '자화상', '어렵게만', '하라', '뛰어난', '훈풍', '카뮈', '혜경궁', '알베르트', '홍씨', '선사', '나가는', '독서', '직접', '책장', '노동', '손', '할', '느껴지던', '리', '사피', '스', '리프킨', '한중록', '초집', '통찰', '시키며', '해', '러미', '남다른', '마음', '인', '설명', '이의', '쉽게', '문서', '꿰뚫는', '구조', '시원하게', '어른', '디테일', '큰']
40-2완료
['책', '독자', '간', '개정', '출', '이야기', '하는', '갖게', '인문학', '나누었던', '대', '바꾼', '한', '않은', 

80-7완료
['케어', '팜', '사례', '운영', '그린비', '하였다', '분석', '기반', '연구', '점', '미시', '하고자', '생태계', '혁신', '목적', '통해', '중간', '별로', '활성화', '도출', '치유', '서비스', '전략', '제시', '단계', '하여', '계층', '거시', '바탕', '취약', '성공', '틀', '실증', '심신', '대한', '영향', '살펴보고', '관점', '개념', '요인', '위', '층', '제공', '한다', '되는', '이용', '특징', '의사', '기법', '프로그램', '이론', '결정', '모델', '전환', '대해', '기술', '사회']
40-4완료
['독서', '법', '퀀텀', '권', '출', '자리', '잡은', '위해', '책', '여러', '돌아왔', '공', '넘어', '바꾼', '배우러', '대한', '시도', '간', '다년', '갈증', '전', '명실', '명확한', '만이', '되었던', '헤매던', '캐나다', '인생', '한계', '찾아', '오는', '했으나', '채우기', '리딩', '달래기', '된', '김병완', '저자', '명의', '주년', '성원', '준비', '아쉬움', '했다', '중국', '수강생', '미미한', '내', '이후', '불러일으켰다', '방법', '작가', '보답', '보며', '미국', '간절함과', '제주', '개정판', '맞아', '더욱', '아니라', '성과', '준', '반향', '획기', '보강', '우리나라', '양성', '더', '안겨', '탓', '큰', '세계', '명', '곳곳', '삶', '새로운', '나은']
0-6완료
['책', '도끼', '저자', '깊이', '하는', '통해', '읽기를', '분야', '다는', '인문학', '교양', '내', '같은', '등장', '증명', '였다', '인문', '읽은', '다양한', '소개', '왔다고', '한다', '있는', '감성', '과학', '

...

60-20완료
['책', '이동진', '부', '작가', '불구', '생각', '그럼에도', '법', '방법', '담은', '권', '갖고', '읽어', '있는', '장서가', '이자', '하는', '읽는', '온', '권하고', '중', '사랑', '읽기가', '목록', '추천', '세상', '도서', '독서', '만', '이야기', '대화', '있고', '일평생', '읽고', '읽을', '읽었고', '애서가', '담고', '빨간', '말리는', '싶은', '책방', '가장', '가려', '기자', '뽑은', '재미있다고', '못', '호흡', '이다혜', '파하는', '맞추었던', '설', '씨', '리스트', '수록', '했다']
31-11완료
['독서', '엄마', '질문', '생각', '남미영', '멘토', '박사', '표', '대한민국', '법', '코칭', '초등', '키우는', '키우기', '놀이', '달라진다', '깊이', '해도', '책임져', '하는', '위', '가정', '마트', '할', '활약', '교육', '아빠', '거리', '주고받는', '자녀', '온', '익혀야', '아동기', '소개', '그림책', '공부', '명작', '하며', '꼭', '유아기', '시기', '읽기까지', '읽어도', '전', '동화', '책', '동시', '위인', '교과서', '아니라', '똑같은']
1-10완료
['권', '다시', '유시민', '책', '청춘', '한', '만든', '세상', '뒤', '시절', '하는', '젊음', '했던', '만난다', '지식', '지식인', '전복', '바꾼', '원천', '꺼내', '든', '읽을', '답', '커버', '소매', '고전', '집어', '소설', '끊임없이', '이르렀', '미래', '이야기', '고뇌', '시켰던', '청년', '한다', '독서', '명예', '들었다', '과거', '잃어버린', '대한', '질문', '살다', '펼친', '지도', '긴', '이정표', '멀', '되어준', 

1-12완료
['채널', '유튜브', '영상', '모으는', '독자', '기획', '수익', '여러', '외', '만들고', '만들었다면', '꾸미고', '찾아', '먼저', '방법', '기억', '덤', '남는', '만', '템플릿', '해', '소재', '내는', '안', '만들기', '메주', '이제', '소', '자막', '공개', '개정', '김', '분야', '만날', '베스트셀러', '유튜버', '줄', '찐', '비밀스러운', '광고', '러시', '쇼츠', '편집', '짧은', '프리미어', '기반', '판', '구조', '십분', '양식', '일차', '홀리', '된다', '설정', '놓치지', '상담', '소개', '알려', '탄탄히', '출시', '챙기자', '법', '필', '다진', '모르면', '독서', '초보', '하는', '볼', '저작권', '활용', '다시', '위', '기술', '알림', '수', '예능', '시청자', '무조건', '손해', '돌아왔다', '한', '만드는', '요즘', '있는', '좋아요', '시작', '만나', '차근차근', '뚝딱', '책', '보자', '편', '가장', '스튜디오', '내', '할', '자문', '최근', '될', '있을까', '크리에이터', '구독', '평범한', '펼쳐', '있다면', '막연한', '지금', '동기부여', '직장인', '아닌', '담았다', '눌러', '소소한', '매일', '꾹꾹', '특별해지기까지', '유튜버로']
61-1완료
['세계사', '퀴즈', '상식', '이야기', '자주', '역사', '용어', '접해', '배우는', '더욱', '은', '재미있는', '재미있게', '고대', '파란만장한', '되다', '공부', '통해', '이어지는', '어린이', '담겨', '보며', '어렵기만', '하지요', '하기', '키워', '배운', '시대', '새롭게', '많습니다', '더', '모험', '흥미진진', '먹는', '중세', '가까워지는', '할', '듣

81-7완료
['책', '저자', '계단', '독자', '사장', '채', '불편한', '질문', '하며', '인문학', '오르는', '지식', '만', '인문', '못', '통해', '내면', '닿지', '반복', '에세이', '권한다', '색다른', '보낸', '아우른다', '마주', '균열', '첫', '읽는', '들은', '나를', '읽고', '한', '삶', '게', '대', '지난', '만나', '문학', '필명', '공부', '올라온', '가장', '온', '시절', '얕', '굽이', '말', '얕은', '고민', '성장', '최전선', '사랑', '한다', '영역', '가깝게', '독서', '밟아', '시킬', '지대', '사로잡은', '종교', '넓', '우물', '나가면', '일으켜', '하는', '지적', '갇힌', '깨달음', '펼쳐진다', '수', '불편하게', '방향', '찌를', '다가', '아니라서', '면', '될', '영혼', '변화', '내다보게', '없는', '청춘', '넓고', '사색', '커다란', '교통사고', '인간', '지평', '인생', '나타나', '살아왔기에', '새로운', '만날', '수도', '예술', '나아가고', '학창', '있다는', '차례', '꼴', '밖', '위', '지냈던', '넓히지만', '있었는지', '가둘', '대화', '철학', '생각', '있다고', '시키는지', '지도', '연히', '오늘', '아닌', '아니라', '했던', '과학', '숫자', '몸소', '경제학', '역사', '홀', '이르', '건', '보여준다', '날', '렀', '요', '중요한', '대단한', '있지만', '독서광', '알려져']
61-3완료
['책', '읽기를', '시기', '부모', '저학년', '한다', '세', '수', '독서', '읽기에', '하는', '이야기', '읽기', '이뤄지지', '할', '줄', '초등', '있는', '만다', '수수방관', '벗어나지', '기를', '해야', '읽기

11-18완료
['지능', '인공', '세계', '살아남기', '오딘', '매력', '되지요', '개념', '기술', '테마', '로봇', '접', '있습니다', '푹', '빠지게', '발전', '활용', '파크', '종류', '홈', '받고', '수준', '떠난', '대해', '스마트', '지지', '지오', '알아보며', '있는', '열렬한', '이야기', '한층', '되어', '해외', '펼쳐집니다', '업그레이드', '곳', '연결', '대만', '고글', '된', '정보', '생생한', '친근하고', '인터넷', '인도네시아', '역', '재미', '쓰고', '사물', '말레이시아', '가상', '적', '하고', '중국', '그림', '여행', '기도', '즐기', '통해', '태국', '만화', '일본', '상식', '만나', '과학', '재미있는', '시리즈', '미국', '보세요', '서바이벌', '효과', '에듀테인먼트', '했고', '전달', '본격', '기록', '평가', '베스트셀러', '받으며', '권']
51-14완료
['고전', '장', '인문', '독서', '위', '법', '교육법', '가이드', '역사', '길러내는', '힘', '저자', '철학', '리더', '안내', '교육', '인생', '세계', '체계', '대표', '성인', '명사', '인문학', '바꾸는', '하는', '자본주의', '도서', '문학', '들을', '도전', '초보자', '배우는', '인류', '단계', '영법', '되는', '총', '쓴', '부모', '리딩', '초등학생', '담고', '필요한', '애서가', '시스템', '개인', '이루어져', '있으며', '운명', '아니라', '말', '싶은', '정리', '초심자', '추천', '개정판', '리드', '승자', '새롭게', '별', '부록', '권말', '나라', '돕기', '해', '실었다', '위해', '가문', '이해', '인터뷰', '진솔한', '이지성', '다락방', '부분', '

31-18완료
['독서', '법', '퀀텀', '권', '출', '자리', '잡은', '위해', '책', '여러', '돌아왔', '공', '넘어', '바꾼', '배우러', '대한', '시도', '간', '다년', '갈증', '전', '명실', '명확한', '만이', '되었던', '헤매던', '캐나다', '인생', '한계', '찾아', '오는', '했으나', '채우기', '리딩', '달래기', '된', '김병완', '저자', '명의', '주년', '성원', '준비', '아쉬움', '했다', '중국', '수강생', '미미한', '내', '이후', '불러일으켰다', '방법', '작가', '보답', '보며', '미국', '간절함과', '제주', '개정판', '맞아', '더욱', '아니라', '성과', '준', '반향', '획기', '보강', '우리나라', '양성', '더', '안겨', '탓', '큰', '세계', '명', '곳곳', '삶', '새로운', '나은']
Error in search_book for the book: 81 - 지적 대화를 위한 넓고 얕은 지식 0 
81-9완료
['사랑', '아낌의', '통해', '공기', '철학', '한', '말', '할', '사람', '주제', '수', '사유', '인문학', '배우다', '문학', '의미', '불교', '강의', '정신', '서양', '필요할', '채워주는', '시대', '진짜', '이끈다', '독자', '키워', '익숙한', '되어', '강신주', '만큼', '자꾸만', '하려면', '드는', '삶', '공허하게', '생', '중요한', '철학자', '해', '있도록', '종횡', '시', '깊이', '해야', '담은', '기반', '대해', '강', '키', '주인', '있어야', '현재', '이야기', '가지', '되', '있는가를', '과거', '단어', '아우르며', '실천', '성찰', '한다', '하는', '되어야', '워드', '전체', '감수성', '고통', '하는가', '연결하며', '다

In [ ]:
df2 = pd.DataFrame(result)
df2.to_excel("keywords_by_book_of_group.xlsx", index=False)

In [ ]:
result

In [ ]:
new_dict = {'error_search' : error_search, 'error_collect' : error_collect}
df3 = pd.DataFrame(new_dict)
df3.to_excel('error_list.xlsx', index = False)

In [ ]:
len(error_search)

In [ ]:
error_search

In [ ]:
len(error_collect)

In [ ]:
error_collect